In [5]:
pip install pypdf

   ---------------------------------------- 0.0/290.4 kB ? eta -:--:--
   ---------------------------------------- 290.4/290.4 kB 9.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain.schema.messages import HumanMessage
from langchain.schema.messages import AIMessage

In [2]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assitant to generate questions and answers"),
    ("user", "{input}")
])

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [5]:
chain = prompt | llm | output_parser

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Big_Mac_Index.pdf")
docs = loader.load_and_split()

In [7]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [29]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [30]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [31]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [32]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [38]:
chat_history = [HumanMessage(content="Given the above conversation, generate a search query to look up to get information relevant to the conversation"), AIMessage(content="Yes!")]
response= retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": '''Generate 12 questions and their answers.These questions should fall into three categories: True or False, Multiple Choice Questions (MCQs), and one-word answers.
    Please ensure each category gets 3 questions.
    True or Flase questions should follow the format : Question -> True/False
    One word answer questions should contain only questions and one answer for example : Question -> Answer.'''
})
answers = response['answer']
print(answers)

Of course! Here are 12 questions and their answers based on the conversation you provided:

True or False Questions:

1. McDonald's uses different commercial strategies in different countries, which can result in huge differences for a product. -> True
2. The relative cost of high-margin products, such as essential pharmaceutical products or cellular telephony, might compare local capacity and willingness to pay as much as relative currency values. -> False (High-margin products may have different prices due to local production and delivery costs, advertising costs, and local market conditions)
3. The price of a Big Mac will be a reflection of its local production and delivery cost, the cost of advertising (considerable in some areas), and most importantly, what the local market will bear – quite different from country to country, and not all a reflection of relative currency values. -> True

Multiple Choice Questions:

1. What is the main reason why McDonald's prices may differ across

In [39]:
with open('answers.txt', 'w') as f:
    f.write(answers)